In [1]:
import torch, torchaudio

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cuda = torch.cuda.is_available()

print(device)
print(cuda)

cuda
True


# Extract Embeddings

In [3]:
import torch, torchaudio

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
cuda = torch.cuda.is_available()


In [5]:
from transformers import Wav2Vec2Model


In [7]:
model_name = "facebook/wav2vec2-large-xlsr-53"
model = Wav2Vec2Model.from_pretrained(model_name)

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2Model: ['project_q.weight', 'project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_hid.weight', 'quantizer.weight_proj.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
filepath = "/home/fred/Projetos/DATASETS/MOS/BRSPEECH_MOS_DATASET_old/wavs/2959_2564_000001.wav"

audio_data, sr = torchaudio.load(filepath)

In [9]:
output = model(audio_data).last_hidden_state

In [10]:
output.shape

torch.Size([1, 509, 1024])

In [12]:
from tqdm import tqdm
import os

metadata_filepath = '/home/fred/Projetos/DATASETS/MOS/VCC2018_MOS_preprocessed/mos_list.txt'
wavs_filepath = '/home/fred/Projetos/DATASETS/MOS/VCC2018_MOS_preprocessed/wav'
with open(metadata_filepath, encoding="utf-8") as f:
  content_file = f.readlines()

output_dir = "./VCC2018_wav2vec_embeddings"
os.makedirs(output_dir, exist_ok=True)

for line in tqdm(content_file):
    #filepath, mos, condition, database = line.split(',')
    filepath, mos = line.split(',')
    filename = os.path.basename(filepath)
    filepath = os.path.join(wavs_filepath, filepath)
    if not os.path.exists(filepath):
      continue
    audio_data, sr = torchaudio.load(filepath)
    audio_data = audio_data.to(device).squeeze()

    audio_data = processor(audio_data, return_tensors="pt", padding=True, sampling_rate=16000).input_values

    # Extract Embedding    
    file_embedding = model(audio_data)#['last_hidden_state']
    
    # Saving embedding
    output_filename = filename.split(".")[0] + ".pt"
    output_filepath = os.path.join(output_dir, output_filename)

    torch.save(file_embedding, output_filepath)  

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20580/20580 [15:53<00:00, 21.59it/s]


In [14]:
model = model.to(device)

# Extract Embeddings from BRSpeech

In [16]:
metadata_filepath = '/home/fred/Projetos/DATASETS/MOS/BRSPEECH_MOS_DATASET/metadata.csv'
wavs_filepath = "/home/fred/Projetos/DATASETS/MOS/BRSPEECH_MOS_DATASET/"
with open(metadata_filepath, encoding="utf-8") as f:
  content_file = f.readlines()

output_dir = "./brspeech_mos_wav2vec_embeddings"

for line in tqdm(content_file):
    filepath, score, condition, database = line.split(",")
    filename = os.path.basename(filepath)
    complete_filepath = os.path.join(wavs_filepath, filepath)
    if not os.path.exists(complete_filepath):
      continue
    audio_data, sr = torchaudio.load(complete_filepath)
    audio_data = audio_data.to(device)
    # Extract Embedding    
    file_embedding = model(audio_data)#['last_hidden_state']
    
    # Saving embedding
    output_filename = filename.split(".")[0] + ".pt"
    output_filepath = os.path.join(output_dir, os.path.dirname(filepath), output_filename)

    os.makedirs(os.path.dirname(output_filepath), exist_ok=True)
    torch.save(file_embedding, output_filepath)  

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2428/2428 [00:23<00:00, 104.29it/s]
